<a href="https://colab.research.google.com/github/Giffy/Mobility_AIBCN/blob/master/3_Dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset creation

Creation of mobile phone density images of Barcelona

##  Connect to Google Drive

In [2]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/GDrive/
Connection to Google drive successful


##  Download data

In [27]:
# Download previously cleaned data from 'Github'
!pip install wget
import wget
url = 'https://raw.githubusercontent.com/Giffy/Mobility_AIBCN/master/dataset/coverageCat.tar.gz'
try:
  filename = wget.download(url)
  print ( 'Dataset downloaded successfully.')

  # Extract tar file
  import tarfile
  tf = tarfile.open( filename )
  tf.extractall()
  
except:
  print ( 'Error downloading dataset.')


Dataset downloaded successfully.


##  Load data

In [0]:
!ls GDrive/My\ Drive
#!cp GDrive/My\ Drive/coverageCat.tar.gz ./
filename = 'coverageCat.tar.gz'
!ls ./

# Extract tar file
import tarfile
#tf = tarfile.open( filename )
#tf.extractall()

In [11]:
import pandas as pd

# load data
df = pd.read_csv('coverageCat.csv')
df.head()

,Activitat,Day,Hour,Latitude,Longitude,Month,Weekday,Year,_id
0,UNKNOWN,17,1,41.385,2.20,10,6,2015,ObjectId(5c23613498a81066a4429c77)
1,IN_VEHICLE,17,1,41.385,2.14,10,6,2015,ObjectId(5c23613498a81066a4429c85)
2,IN_VEHICLE,17,1,41.385,2.14,10,6,2015,ObjectId(5c23613498a81066a4429c8b)
3,IN_VEHICLE,17,2,41.385,2.14,10,6,2015,ObjectId(5c23613498a81066a4429c90)
4,UNKNOWN,17,2,41.390,2.20,10,6,2015,ObjectId(5c23613498a81066a4429ca0)


In [12]:
def round_coord(x, base=0.005):
     return round(base*round(x/base), 3)

df['Latitude'] = df['Latitude'].apply(round_coord)
df['Longitude_fix'] = df['Longitude'].apply(round_coord)

df.head()

,Activitat,Day,Hour,Latitude,Longitude,Month,Weekday,Year,_id,Longitude_fix
0,UNKNOWN,17,1,41.385,2.20,10,6,2015,ObjectId(5c23613498a81066a4429c77),2.20
1,IN_VEHICLE,17,1,41.385,2.14,10,6,2015,ObjectId(5c23613498a81066a4429c85),2.14
2,IN_VEHICLE,17,1,41.385,2.14,10,6,2015,ObjectId(5c23613498a81066a4429c8b),2.14
3,IN_VEHICLE,17,2,41.385,2.14,10,6,2015,ObjectId(5c23613498a81066a4429c90),2.14
4,UNKNOWN,17,2,41.390,2.20,10,6,2015,ObjectId(5c23613498a81066a4429ca0),2.20


##  Generate dataset graphics 



In [0]:
# Sample of plotting graphic with seaborn
# plot of Barcelona the 11th September of 2015

df0 = df[ (df['Year']== 2015) & (df['Month']== 9) & (df['Day']== 11)]
df0 = df0.groupby(['Latitude','Longitude'], as_index= False).count()           
result = df0.pivot (index='Latitude' , columns='Longitude', values='Day')

import seaborn
import seaborn as sns
sns_plot = seaborn.heatmap(result) #, cmap = 'gray')
fig = sns_plot.get_figure()
fig.savefig("output.png")

In [0]:
# Iteration to create the dataset of graphics
years = [2015, 2016, 2017]
months = [1,2,3,4,5,6,7,8,9,10,11,12]
days = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
weekdays = [1,2,3,4,5]
manana = [7,8,9,10,11]
tarde = [16,17,18,19,20]

hours = manana

df0 = df[df['Activitat']=='IN_VEHICLE']
df0 = df0[(df0['Weekday']!=6) & (df0['Weekday']!=7)]
df0 = df0[(df0['Hour'].isin(hours))]

for year in years:  
  df2 = df0[df0['Year']==year]  
  for month in months:
    df3 = df2[df2['Month']==month]
    for day in days:
          df4 = df3[df3['Day']==day]
          df5 = df4.groupby(['Latitude','Longitude'], as_index= False).count()           
          result = df5.pivot (index='Latitude' , columns='Longitude', values='Day')
          try:
            sns_plot = seaborn.heatmap(result, cmap = 'gray')
            fig = sns_plot.get_figure()
            if 10 in hours:
              periodo = 'M'
            else:
              periodo = 'T'
            filename = str(year) + '_' + str(month) + '_' + str(day) + '_' + periodo + '.png' 
            fig.savefig( filename )
          except:
            pass


## Plotting data sample with Folium

In [15]:
!pip install folium geopandas             # Install folium for map visualization

import folium                             # Library to create the map
from folium import plugins                # plugin to create a heatmap

    100% |████████████████████████████████| 901kB 19.6MB/s 
    100% |████████████████████████████████| 1.5MB 11.2MB/s 
    100% |████████████████████████████████| 4.4MB 5.1MB/s 
    100% |████████████████████████████████| 17.9MB 1.5MB/s 
  Running setup.py bdist_wheel for pyproj ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/89/77/ec/a537585d1022dafde0317dd19d33c4a30d4ee61e19f25ebd8e
  Running setup.py bdist_wheel for munch ... - done
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built pyproj 

In [0]:
df0 = df[ (df['Year']== 2015) & (df['Month']== 9) & (df['Day']== 11)]
df0 = df0.groupby(['Latitude','Longitude'], as_index= False).count()           

latitude = df0.loc [ : , 'Latitude']
longitude = df0.loc [ : , 'Longitude']
total = df0.loc [ : , 'Day']

In [19]:
center_lat = 41.42
center_lon = 2.18

locationmap = folium.Map(location=[ center_lat , center_lon ], zoom_start=12) #, width=800, height=600 )
locationmap.add_child(plugins.HeatMap(zip(latitude, longitude, total), radius = 12))

locationmap